In [1]:
import pdb
import numpy as np
import gcp.bigquery as bq
import gcp.storage as storage
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.optimizers import SGD
import tensorflow as tf
import h5py
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Activation
import time
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
import matplotlib.pyplot as plt

try:
  import cPickle as pickle
except:
  import pickle
HDF_FILENAME = 'best_model.hdf5'
EST_FILENAME = 'final_estimator.pkl'

seed = 13
np.random.seed(seed)

Using TensorFlow backend.


# 1. Load Data

In [2]:
# Put all categorical data first for easier implementation of One Hot Encoding.
fields_str = """
gap	day_in_week	weather_1_slots_ago	weather_2_slots_ago	weather_3_slots_ago	busy_time	
tj_level1_1_slots_ago	tj_level2_1_slots_ago	tj_level3_1_slots_ago	tj_level4_1_slots_ago	
tj_level1_2_slots_ago	tj_level2_2_slots_ago	tj_level3_2_slots_ago	tj_level4_2_slots_ago	
tj_level1_3_slots_ago	tj_level2_3_slots_ago	tj_level3_3_slots_ago	tj_level4_3_slots_ago	
temperature_1_slots_ago	pm25_1_slots_ago	
temperature_2_slots_ago	pm25_2_slots_ago	
temperature_3_slots_ago	pm25_3_slots_ago	
gap_1_slots_ago	sum_price_1_slots_ago	
gap_2_slots_ago	sum_price_2_slots_ago	
gap_3_slots_ago	sum_price_3_slots_ago	
f1	f11	f11_1	f11_2	f11_3	f11_4	f11_5	f11_6	f11_7	
f11_8	f13_4	f13_8	f14	f14_1	f14_10	f14_2	f14_3	f14_6	f14_8	f15	f15_1	
f15_2	f15_3	f15_4	f15_6	f15_7	f15_8	f16	f16_1	f16_10	f16_11	f16_12	f16_3	
f16_4	f16_6	f17	f17_2	f17_3	f17_4	f17_5	f19	f19_1	f19_2	f19_3	f19_4	f1_1	
f1_10	f1_11	f1_2	f1_3	f1_4	f1_5	f1_6	f1_7	f1_8	f20	f20_1	f20_2	
f20_4	f20_5	f20_6	f20_7	f20_8	f20_9	f21_1	f21_2	f22	f22_1	f22_2	f22_3	
f22_4	f22_5	f23	f23_1	f23_2	f23_3	f23_4	f23_5	f23_6	f24	f24_1	f24_2	f24_3	
f25	f25_1	f25_3	f25_7	f25_8	f25_9	f2_1	f2_10	f2_11	f2_12	f2_13	f2_2	
f2_4	f2_5	f2_6	f2_7	f2_8	f3_1	f3_2	f3_3	f4	f4_1	f4_10	f4_11	
f4_13	f4_14	f4_16	f4_17	f4_18	f4_2	f4_3	f4_5	f4_6	f4_7	f4_8	f4_9	
f5	f5_1	f5_3	f5_4	f6	f6_1	f6_2	f6_4	f7	f8	f8_1	f8_2	f8_3	f8_4	
f8_5
"""
fields = map(lambda x: x.strip(), fields_str.split('\t'))
features = fields[1:]

# Use this instead of len(features) since this variable can change
# e.g. when one hot encoding is used and/or new features are added.
n_features = len(features)

In [3]:
%%sql --module q_all

SELECT *, HASH(CAST(district_id AS STRING) +timeslot) AS hash_value,
  IF(ABS(HASH(CAST(district_id AS STRING) + timeslot)) % 2 == 1, 'True', 'False')
    AS included_in_sample, IF(timeofday_slot >= 50 AND timeofday_slot <= 53, 1, 0) AS busy_time
FROM [datalab-projects-1331:xjk_algo_comp.gaps]
WHERE gap > 0
LIMIT 5000

# The above query randomizes its outputs.

In [4]:
query = bq.Query(q_all)
tableresult = query.results()

all_data = np.zeros((tableresult.length, len(fields)))
print 'there are {} rows'.format(tableresult.length)
for rcounter, row in enumerate(tableresult):
  for fcounter, field in enumerate(fields):
    all_data[rcounter, fcounter] = row[field]
  if rcounter % 5000 == 0:
    print 'processed {} rows'.format(rcounter)
all_data_original = np.copy(all_data)

there are 5000 rows
processed 0 rows


# 2. Preprocess Data

Note that in the end we don't use data adjusted from Preprocessing steps, but rather including the preprocessing instances in Pipeline to be applied to original data.

In [5]:
all_data = np.copy(all_data_original)

## 2.1. Impute NaN and inf values

In [6]:
from sklearn.preprocessing import Imputer
# Useful code to check NaN and Inf values. This is needed since these values would
# cause "Input contains NaN, infinity or a value too large for dtype('float32')
# errors when left unchecked.
print "Checkinf for NaN and Inf"
print "np.nan=", np.where(np.isnan(all_data))
print "is.inf=", np.where(np.isinf(all_data))
print "np.max=", np.max(abs(all_data))

# Impute
imputer = Imputer()
imputer.fit(all_data)
print 'Shape of old data:'
print all_data.shape
print 'Stats (NaN values are replaced with following means for each feature):'
print imputer.statistics_
all_data = imputer.fit_transform(all_data)
print 'Shape of new data:'
print all_data.shape

# See that NaN and Inf values replaced
print "Checkinf for NaN and Inf"
print "np.nan=", np.where(np.isnan(all_data))
print "is.inf=", np.where(np.isinf(all_data))
print "np.max=", np.max(abs(all_data))

Checkinf for NaN and Inf
np.nan= (array([  25,   25,   32, ..., 4993, 4993, 4993]), array([24, 25,  6, ..., 21, 22, 23]))
is.inf= (array([], dtype=int64), array([], dtype=int64))
np.max= nan
Shape of old data:
(5000, 164)
Stats (NaN values are replaced with following means for each feature):
[  1.27114000e+01   2.28960000e+00   3.59600786e+00   3.47742985e+00
   3.39274095e+00   0.00000000e+00   9.88235205e+02   2.06198723e+02
   6.17592217e+01   4.31093433e+01   9.91365626e+02   2.09248750e+02
   6.25887042e+01   4.35133090e+01   9.95095055e+02   2.11891096e+02
   6.34207540e+01   4.39057560e+01   4.22648095e+00   1.54515216e+02
   4.22963264e+00   1.52985462e+02   4.24715331e+00   1.52567067e+02
   1.28356049e+01   1.34699639e+03   1.30353461e+01   1.36496606e+03
   1.37977339e+01   1.39147560e+03   1.69990640e+03   7.40665440e+03
   1.32121060e+03   3.25740140e+03   3.74164000e+03   2.21915274e+04
   1.08867780e+03   8.22478540e+03   1.67816040e+03   2.05651258e+04
   1.89839758e+04

## 2.2. One Hot Encoding

In [7]:
one_hot = OneHotEncoder(categorical_features=[1, 2, 3, 4], n_values='auto')
one_hot.fit(all_data)
print "n_values_:"
print one_hot.n_values_
print "feature_indices_:"
print one_hot.feature_indices_
all_data = one_hot.fit_transform(all_data).todense()
n_features = all_data.shape[1] - 1
print 'new number of features: {}'.format(n_features)

n_values_:
[ 7 10 10 10]
feature_indices_:
[ 0  7 17 27 37]
new number of features: 189


## 2.3. Feature Scaling

In [8]:
scaler = StandardScaler()
print 'old data:'
print 'means: {}, min: {}, max: {}'.format(np.mean(all_data), np.min(all_data), np.max(all_data))
all_data = scaler.fit_transform(all_data)
print 'new data:'
print 'means: {}, min: {}, max: {}'.format(np.mean(all_data), np.min(all_data), np.max(all_data))

old data:
means: 2630.44168786, min: 0.0, max: 461563.0
new data:
means: -1.22063762606e-17, min: -1.94267747305, max: 20.7964022574


# 3. Split into Train and Test Data

In [9]:
# Split the originaldata into train and test sets.
data_size = all_data_original.shape[0]
training_size = data_size * 90/100
indices = np.random.permutation(data_size)
training_idx, test_idx = indices[:training_size], indices[training_size:]
all_data_train, all_data_test = all_data_original[training_idx,:], all_data_original[test_idx,:]

data_train = all_data_train[:,1:]
targets_train = all_data_train[:,0]
data_test = all_data_test[:,1:]
targets_test = all_data_test[:,0]

# 4. Model Creation

## 4.1. Loss Function

In [10]:
def mape(y, predictions):
  return K.mean(K.abs(y-predictions/K.clip(K.abs(y), K.epsilon(), np.inf)), axis=-1)

## 4.2. Create Pipeline

In [ ]:
def create_model(optimizer='adagrad', init='uniform'):
  model = Sequential()
  model.add(Dense(n_features, input_dim=n_features, init=init, activation='relu'))
  model.add(Dense(floor(n_features/2), init=init, activation='relu'))
  model.add(Dense(floor(n_features/4), init=init, activation='relu'))
  model.add(Dense(1))
  model.compile(loss=mape, optimizer=optimizer)
  return model

model = KerasRegressor(build_fn=create_model, nb_epoch=20, batch_size=10, verbose=2)

steps = [
  ('impute', Imputer()),
  # Remember that gap, the first variable, is removed, thus categorical_features
  # should start at index 0.
  ('one_hot', OneHotEncoder(categorical_features=[0, 1, 2, 3], sparse=False)),
  ('scale', StandardScaler()),
#   ('pca', PCA(n_components=120)),
  ('estimate', model)
]
final_est = Pipeline(steps)

# kfold = StratifiedKFold(y=targets_train, n_folds=10, shuffle=True, random_state=seed)

## 4.3. Grid Search to Find Best Hyperparameters

Use subset of training data in grid search just to find what hyperparameters are best for this project. Specifically we want to find the best optimizer, initializer, and batch size. Topology of the model will be adjusted in crossvalidation step.

In [ ]:
# model.fit(data_train, targets_train, validation_split=0.33,  nb_epoch=100, batch_size=10, 
#           callbacks=callbacks_list)
# results = cross_val_score(model, data_train, targets_train, cv=kfold)

checkpoint = ModelCheckpoint(HDF_FILENAME, monitor='loss', save_best_only=True, mode='min')
callbacks_list = [checkpoint]

epochs = 150
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

data_train_s = data_train[:1000]
targets_train_s = targets_train[:1000]
optimizers = [sgd, 'adam', 'adagrad']
init = ['glorot_uniform', 'normal', 'uniform']
epochs_r = np.array([epochs])
batches = np.array([5, 10, 20])
param_grid = {
  'estimate__optimizer': optimizers,
  'estimate__nb_epoch': epochs_r, 
  'estimate__batch_size': batches,
  'estimate__init': init,
  'estimate__callbacks': [callbacks_list]
}

grid = GridSearchCV(estimator=final_est, param_grid=param_grid, cv=3)
grid_result = grid.fit(data_train_s, targets_train_s)
time.sleep(0.5)

2s - loss: 4.8714 - acc: 0.3103
Epoch 113/150
2s - loss: 4.8808 - acc: 0.3043
Epoch 114/150
2s - loss: 4.8042 - acc: 0.3118
Epoch 115/150
2s - loss: 4.8054 - acc: 0.3163
Epoch 116/150
2s - loss: 4.7771 - acc: 0.3118
Epoch 117/150
2s - loss: 4.7390 - acc: 0.3088
Epoch 118/150
2s - loss: 4.7882 - acc: 0.3118
Epoch 119/150
2s - loss: 4.7683 - acc: 0.3103
Epoch 120/150
2s - loss: 4.7966 - acc: 0.3073
Epoch 121/150
2s - loss: 4.7156 - acc: 0.3073
Epoch 122/150
2s - loss: 4.7112 - acc: 0.3163
Epoch 123/150
2s - loss: 4.7374 - acc: 0.3073
Epoch 124/150
2s - loss: 4.7037 - acc: 0.3073
Epoch 125/150
2s - loss: 4.7448 - acc: 0.3088
Epoch 126/150
2s - loss: 4.7367 - acc: 0.3148
Epoch 127/150
2s - loss: 4.6823 - acc: 0.3088
Epoch 128/150
2s - loss: 4.7182 - acc: 0.3163
Epoch 129/150
2s - loss: 4.6655 - acc: 0.3088
Epoch 130/150
2s - loss: 4.7009 - acc: 0.3073
Epoch 131/150
2s - loss: 4.6063 - acc: 0.3178
Epoch 132/150
2s - loss: 4.6890 - acc: 0.3028
Epoch 133/150
2s - loss: 4.6766 - acc: 0.3118
Ep

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for params, mean_score, scores in grid_result.grid_scores_:
print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

In [ ]:
scores = model.evaluate(data_train, targets_train)
print("\nscore: %.2f%" % (scores))

In [ ]:
scores = model.evaluate(data_test, targets_test)
print("\nscore: %.2f%" % (scores))

## 4.4. Crossvalidation

Use entire training data for cross validation. Resulting model from here will be the final model.

In [ ]:
epochs = 500
def create_final_model():
  learning_rate = 1
  decay_rate = learning_rate / epochs
  momentum = 0.8
  sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
  
  final_model = Sequential()
  final_model.add(Dropout(0.2, input_shape=(n_features,)))
  final_model.add(Dense(n_features, init=init, activation='relu'))
  final_model.add(Dropout(0.2))
  final_model.add(Dense(floor(n_features/2), init=init, activation='relu'))
  final_model.add(Dropout(0.2))
  final_model.add(Dense(floor(n_features/4), init=init, activation='relu'))
  final_model.add(Dropout(0.2))
  final_model.add(Dense(1))
  final_model.compile(loss=mape, optimizer=optimizer)

model = KerasRegressor(build_fn=create_model, nb_epoch=20, batch_size=10, verbose=2)

steps = [
  ('impute', Imputer()),
  # Remember that gap, the first variable, is removed, thus categorical_features
  # should start at index 0.
  ('one_hot', OneHotEncoder(categorical_features=[0, 1, 2, 3], sparse=False)),
  ('scale', StandardScaler()),
#   ('pca', PCA(n_components=120)),
  ('estimate', final_model)
]
final_est = Pipeline(steps)

history = final_est.fit(data_train, targets_train, validation_split=0.33,
                        nb_epoch=150, batch_size=10, verbose=0)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

kfold = KFold(n=len(data_train), n_folds=10, random_state=seed)
results = cross_val_score(estimator, data_train, targets_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
predictions = model.predict(data_test)

In [ ]:
print(predictions[0:100])

# Store Model to File

In [ ]:
model_json = model.to_json()
with open("final_model.json", "w") as json_file:
  json_file.write(model_json)
  
# serialize weights to HDF5
model.save_weights("final_model.h5")
print("Saved model to disk")

In [ ]:
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss=mape, optimizer='adam')
score = loaded_model.evaluate(X, Y, verbose=0)
print "%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100)
